In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"

import pathlib
import glob
from copy import deepcopy

import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import jax
jax.config.update("jax_platform_name", "cpu")
# jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp
import equinox as eqx

In [ ]:
from mc2.utils.data_inspection import (
    get_available_material_names,
    get_file_overview,
    load_and_process_single_from_full_file_overview
)
from mc2.data_management import load_data_into_pandas_df, FrequencySet, MaterialSet, DataSet, DATA_ROOT, FINAL_MATERIALS

In [ ]:
FINAL_MATERIALS

In [ ]:
material_sets = []

for material_name in FINAL_MATERIALS:
    data_dict = load_data_into_pandas_df(material=material_name)
    material_sets.append(MaterialSet.from_pandas_dict(data_dict))

In [ ]:
dataset = DataSet(material_sets)

In [ ]:
number = 0

for material_set in dataset:
    for frequency_set in material_set:
        number += frequency_set.H.size

print(number)

# Misc:

In [ ]:
from mc2.features.features_jax import dyn_avg, d2b_dt2, db_dt

### A at higher frequencies very noisy. Why?

In [ ]:
subset = dataset.at_material("A").at_frequency(800_000)
for seq_H, seq_B in zip(subset.H[:1, :400], subset.B[:1, :400]):
    for n_s in [1, 5, 11, 21, 51, 101]:
        fig, axs = plt.subplots(1,2, figsize=(9,4))
        axs[0].plot(dyn_avg(seq_H, n_s=n_s), c="tab:blue", label="H_dyn_average")
        axs[0].plot(seq_H, c="tab:blue", alpha=0.3, label="H")
        axs[1].plot(seq_B, c="tab:orange", label="B")
        for ax in axs:
            ax.grid(True, alpha=0.3)
            ax.legend()
            ax.set_xlabel("k")
        plt.show()

In [ ]:
subset = dataset.at_material("A").at_frequency(320_000)
for seq_H, seq_B in zip(subset.H[:1, 150:225], subset.B[:1, 150:225]):
    for n_s in [1, 5, 11, 21, 51, 101]:
        fig, axs = plt.subplots(1,2, figsize=(9,4))
        # axs[0].plot(dyn_avg(seq_H, n_s=n_s), c="tab:blue", label="H_dyn_average")
        axs[0].plot(seq_H / jnp.max(jnp.abs(seq_H)), c="tab:blue", alpha=0.7, label="H")
        axs[0].plot(db_dt(seq_B) / jnp.max(jnp.abs(db_dt(seq_B))), c="tab:red", alpha=0.7, label="dbdt")
        axs[0].plot(d2b_dt2(seq_B) / jnp.max(jnp.abs(d2b_dt2(seq_B))) , c="tab:orange", alpha=0.7, label="d2bdt2")

        axs[1].plot(seq_B, c="tab:orange", label="B")
        for ax in axs:
            ax.grid(True, alpha=0.3)
            ax.legend()
            ax.set_xlabel("k")
        plt.show()

In [ ]:
subset = dataset.at_material("B").at_frequency(800_000)
mu_0 = 4 * jnp.pi * 1e-7

H = jnp.diff(subset.H[0])
B = jnp.diff(subset.B[0])

mu = B / (H * mu_0)
mu = mu[jnp.abs(mu) < 10_000]
mu = mu[mu > 0]
plt.hist(mu, bins=100);
#plt.ylim(0, 10_000)

In [ ]:
subset = dataset.at_material("E").at_frequency(320_000)
mu_0 = 4 * jnp.pi * 1e-7

H = jnp.diff(subset.H[0])
B = jnp.diff(subset.B[0])

mu = B / (H * mu_0)
mu = mu[jnp.abs(mu) < 10_000]
mu = mu[mu > 0]
plt.hist(mu, bins=100);

In [ ]:
subset = dataset.at_material("A").at_frequency(800_000)
mu_0 = 4 * jnp.pi * 1e-7

H = jnp.diff(subset.H[0])
B = jnp.diff(subset.B[0])

mu = B / (H * mu_0)
mu = mu[jnp.abs(mu) < 10_000]
mu = mu[mu > 0]
plt.hist(mu, bins=100);
#plt.ylim(0, 10_000)


### C:

In [ ]:
for freq_set in dataset.at_material("C"):
    for seq_H, seq_B in zip(freq_set.H[:1], freq_set.B[:1]):
        fig, axs = plt.subplots(1,2, figsize=(9,4))

        fig.suptitle(freq_set.frequency)
        
        axs[0].plot(seq_H, c="tab:blue", label="H")
        axs[1].plot(seq_B, c="tab:orange", label="B")
        for ax in axs:
            ax.grid(True, alpha=0.3)
            ax.legend()
            ax.set_xlabel("k")
        plt.show()
    
    
        fig, ax = plt.subplots(1,1, figsize=(9,9))
        ax.plot(seq_H, seq_B, c="tab:blue")
        ax.grid(True, alpha=0.3)
        ax.set_xlabel("H")
        ax.set_ylabel("B")

### Weird behavior of D overall? Should E not be the weird one?

In [ ]:
subset = dataset.at_material("D").at_frequency(50_000)

In [ ]:
for seq_H, seq_B in zip(subset.H[:2], subset.B[:2]):
    fig, axs = plt.subplots(1,2, figsize=(9,4))
    axs[0].plot(seq_H, c="tab:blue", label="H")
    axs[1].plot(seq_B, c="tab:orange", label="B")
    for ax in axs:
        ax.grid(True, alpha=0.3)
        ax.legend()
        ax.set_xlabel("k")
    plt.show()


    fig, ax = plt.subplots(1,1, figsize=(9,9))
    ax.plot(seq_H, seq_B, c="tab:blue")
    ax.grid(True, alpha=0.3)
    ax.set_xlabel("H")
    ax.set_ylabel("B")

### E saturation strong for low and high frequencies, but not in between?

In [ ]:
subset = dataset.at_material("E")
subset

In [ ]:
for freq_set in subset:
    for seq_H, seq_B in zip(freq_set.H[:1], freq_set.B[:1]):
        fig, axs = plt.subplots(1,2, figsize=(9,4))

        fig.suptitle(freq_set.frequency)
        
        axs[0].plot(seq_H, c="tab:blue", label="H")
        axs[1].plot(seq_B, c="tab:orange", label="B")
        for ax in axs:
            ax.grid(True, alpha=0.3)
            ax.legend()
            ax.set_xlabel("k")
        plt.show()
    
    
        fig, ax = plt.subplots(1,1, figsize=(9,9))
        ax.plot(seq_H, seq_B, c="tab:blue")
        ax.grid(True, alpha=0.3)
        ax.set_xlabel("H")
        ax.set_ylabel("B")

# EDA:

## Overview:

### Datapoints per material:

In [ ]:
datapoints_per_material = {material_name: 0 for material_name in dataset.material_names}
for idx, material_set in enumerate(dataset):
    for frequency_set in material_set:
        assert frequency_set.H.shape ==  frequency_set.B.shape
        datapoints_per_material[frequency_set.material_name] += frequency_set.H.size

fig, ax = plt.subplots(figsize=(4, 4), dpi=150)
ax.pie(list(datapoints_per_material.values()), labels=list(datapoints_per_material.keys()))
ax.set_title("Size portions per material");

### Datapoints per frequency:

In [ ]:
datapoints_per_frequency = {frequency: 0 for frequency in dataset[0].frequencies.tolist()}
for idx, material_set in enumerate(dataset):
    for frequency_set in material_set:
        datapoints_per_frequency[frequency_set.frequency] += frequency_set.H.size

fig, ax = plt.subplots(figsize=(4, 4), dpi=150)
ax.pie(list(datapoints_per_frequency.values()), labels=list(datapoints_per_frequency.keys()))
ax.set_title("Size portions per frequency in kHz");

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(20, 6), sharey=True, sharex=True, squeeze=False)

fig.suptitle("Datapoints per frequency per material")

for idx, material_set in enumerate(dataset):
    row_idx = 0 if idx < 5 else 1
    col_idx = idx % 5
    
    full_set_lengths = []
    for freq_set in material_set:
        n_sequences, sequence_length = freq_set.B.shape
        full_set_length = n_sequences * sequence_length
        full_set_lengths.append(full_set_length)

    axs[row_idx, col_idx].plot(material_set.frequencies / 1e3, np.array(full_set_lengths) / 1e6)
    axs[row_idx, col_idx].set_title(material_set.material_name)
    
for ax in axs[-1]:
    ax.set_xlabel("f in kHz")

for ax in axs[:, 0]:
    ax.set_ylabel("# of datapoints in M")
        
for ax_ in axs:
    for ax in ax_:
        ax.grid()

fig.tight_layout()
plt.show()

### Datapoints per temperature:

In [ ]:
datapoints_per_temperature = {temperature: 0.0 for temperature in jnp.unique(dataset[0][0].T).tolist()}
for idx, material_set in enumerate(dataset):
    for frequency_set in material_set:
        for temperature in datapoints_per_temperature.keys():
            datapoints_per_temperature[temperature] += jnp.sum(frequency_set.H[jnp.where(frequency_set.T == temperature)[0]].size)

fig, ax = plt.subplots(figsize=(4, 4), dpi=150)
ax.pie(list(datapoints_per_temperature.values()), labels=list(datapoints_per_temperature.keys()))
ax.set_title("Size portions per Temperature in °C");

In [ ]:
print("unique temperatures per frequency")
unique_temperatures = {int(frequency): jnp.unique(freq_set.T) for frequency, freq_set in zip(material_set.frequencies, material_set)}
display(unique_temperatures)

for freq_set in material_set:
    plt.suptitle("Temperatures per sequence")
    plt.plot(freq_set.T, label=str(int(freq_set.frequency / 1e3)) + " kHz")
    plt.legend()

plt.xlabel("sequence index")
plt.ylabel("T in °C")
plt.grid()
plt.show()

### H Histogram per material:

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(20, 6), squeeze=False)
fig.suptitle("H histogram per material")

for idx, material_set in enumerate(dataset):
    row_idx = 0 if idx < 5 else 1
    col_idx = idx % 5

    H_values = jnp.concatenate([freq_set.H.flatten() for freq_set in material_set])
    axs[row_idx, col_idx].hist(H_values, bins=50)
    axs[row_idx, col_idx].set_title(material_set.material_name)
    
for ax in axs[-1]:
    ax.set_xlabel("H in A/m")

for ax in axs[:, 0]:
    ax.set_ylabel("# of datapoints")

for ax_ in axs:
    for ax in ax_:
        ax.grid()

fig.tight_layout()
plt.show()

Fairly poor distribution considering high valued $H(t)$. How should one deal with this? Likely models will struggle in these value ranges?

### B Histogram per material:

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(20, 6), squeeze=False)
fig.suptitle("B histogram per material")

for idx, material_set in enumerate(dataset):
    row_idx = 0 if idx < 5 else 1
    col_idx = idx % 5

    B_values = jnp.concatenate([freq_set.B.flatten() for freq_set in material_set])
    axs[row_idx, col_idx].hist(B_values, bins=50)
    axs[row_idx, col_idx].set_title(material_set.material_name)
    
for ax in axs[-1]:
    ax.set_xlabel("B in Vs/m^2")

for ax in axs[:, 0]:
    ax.set_ylabel("# of datapoints")

for ax_ in axs:
    for ax in ax_:
        ax.grid()

fig.tight_layout()
plt.show()

## Main Loop Hysteresis plots:

### Single example:

In [ ]:
# BH curves

fig, axs = plt.subplots(1, 5, figsize=(20, 6), squeeze=False)
fig.suptitle("Exemplary hysteresis per material for 50 kHz")

for idx, material_set in tqdm.tqdm(enumerate(dataset)):
    row_idx = 0 if idx < 5 else 1
    col_idx = idx % 5

    B_values = material_set[0].B[0, :]
    H_values = material_set[0].H[0, :]
    axs[row_idx, col_idx].plot(H_values, B_values)
    axs[row_idx, col_idx].set_title(material_set.material_name)
    
for ax in axs[-1]:
    ax.set_ylabel("B in Vs/m^2")

for ax in axs[:, 0]:
    ax.set_xlabel("H in A/m")

for ax_ in axs:
    for ax in ax_:
        ax.grid()

fig.tight_layout()
plt.show()

In [ ]:
# HB curves

fig, axs = plt.subplots(1, 5, figsize=(20, 6), squeeze=False)
fig.suptitle("Exemplary hysteresis per material for 50 kHz")

for idx, material_set in tqdm.tqdm(enumerate(dataset)):
    row_idx = 0 if idx < 5 else 1
    col_idx = idx % 5

    B_values = material_set[0].B[125, :]
    H_values = material_set[0].H[125, :]
    axs[row_idx, col_idx].plot(H_values, B_values)
    axs[row_idx, col_idx].set_title(material_set.material_name)
    
for ax in axs[0, :]:
    ax.set_ylabel("B in Vs/m^2")
    ax.set_xlabel("H in A/m")    

for ax_ in axs:
    for ax in ax_:
        ax.grid()

fig.tight_layout()
plt.show()

### All over each other:

In [ ]:
# HB curves

fig, axs = plt.subplots(1, 5, figsize=(20, 6), squeeze=False)
fig.suptitle("Hysteresis per material")

for idx, material_set in tqdm.tqdm(enumerate(dataset)):
    row_idx = 0 if idx < 5 else 1
    col_idx = idx % 5
    
    for frequency_set in material_set: 
        for sequence_idx in range(frequency_set.B.shape[0]):
            B_values = frequency_set.B[sequence_idx, :]
            H_values = frequency_set.H[sequence_idx, :]
            axs[row_idx, col_idx].plot(H_values, B_values, alpha=.7)

    axs[row_idx, col_idx].set_title(material_set.material_name)
    
for ax in axs[0, :]:
    ax.set_ylabel("B in Vs/m^2")
    ax.set_xlabel("H in A/m") 

for ax_ in axs:
    for ax in ax_:
        ax.grid()

fig.tight_layout()
plt.show()

### All over each other, one frequency at a time:

In [ ]:
# HB curves

for frequency in tqdm.tqdm(dataset[0].frequencies):
    fig, axs = plt.subplots(1, 5, figsize=(20, 6), squeeze=False)
    fig.suptitle("Hysteresis per material")
    
    for idx, material_set in enumerate(dataset):
        row_idx = 0 if idx < 5 else 1
        col_idx = idx % 5

        try:
            frequency_set = material_set.at_frequency(frequency)
        except IndexError:
            continue
        for sequence_idx in range(frequency_set.B.shape[0]):
            B_values = frequency_set.B[sequence_idx, :]
            H_values = frequency_set.H[sequence_idx, :]
            axs[row_idx, col_idx].plot(H_values, B_values, alpha=.7)
    
        axs[row_idx, col_idx].set_title(material_set.material_name + " at " + str(frequency_set.frequency / 1e3) + " kHz")

    for ax in axs[0, :]:
        ax.set_ylabel("B in Vs/m^2")
        ax.set_xlabel("H in A/m") 
    
    for ax_ in axs:
        for ax in ax_:
            ax.grid()
    
    fig.tight_layout()
    plt.show()

### Split by temperatures:

In [ ]:
# HB curves

for temperature in [25, 50, 70]:
    for frequency in tqdm.tqdm(dataset[0].frequencies):
        fig, axs = plt.subplots(1, 5, figsize=(20, 6), squeeze=False)
        fig.suptitle(f"Hysteresis per material at {temperature} °C")
        
        for idx, material_set in enumerate(dataset.filter_temperatures(temperature)):
            row_idx = 0 if idx < 5 else 1
            col_idx = idx % 5
            
            try:
                frequency_set = material_set.at_frequency(frequency) 
            except IndexError:
                continue

            for sequence_idx in range(frequency_set.B.shape[0]):
                B_values = frequency_set.B[sequence_idx, :]
                H_values = frequency_set.H[sequence_idx, :]
                axs[row_idx, col_idx].plot(H_values, B_values, alpha=.7)
        
            axs[row_idx, col_idx].set_title(material_set.material_name + " at " + str(frequency_set.frequency / 1e3) + " kHz")

        for ax in axs[0, :]:
            ax.set_ylabel("B in Vs/m^2")
            ax.set_xlabel("H in A/m") 
        
        for ax_ in axs:
            for ax in ax_:
                ax.grid()
        
        fig.tight_layout()
        plt.show()

### Normalized by maximum sequence values:

In [ ]:
# HB curves

for frequency in tqdm.tqdm(dataset[0].frequencies):
    fig, axs = plt.subplots(1, 5, figsize=(20, 6), squeeze=False)
    fig.suptitle(f"Hysteresis per material")
    
    for idx, material_set in enumerate(dataset):
        row_idx = 0 if idx < 5 else 1
        col_idx = idx % 5

        try:
            frequency_set = material_set.at_frequency(frequency) 
        except IndexError:
            continue

        for sequence_idx in range(frequency_set.B.shape[0]):
            B_values = frequency_set.B[sequence_idx, :]
            H_values = frequency_set.H[sequence_idx, :]
            axs[row_idx, col_idx].plot(H_values / jnp.max(jnp.abs(H_values)), B_values / jnp.max(jnp.abs(B_values)), alpha=.7)
    
        axs[row_idx, col_idx].set_title(material_set.material_name + " at " + str(frequency_set.frequency / 1e3) + " kHz")
        
    for ax in axs[0, :]:
        ax.set_ylabel("B in Vs/m^2")
        ax.set_xlabel("H in A/m") 

    for ax_ in axs:
        for ax in ax_:
            ax.grid()
    
    fig.tight_layout()
    plt.show()

## Local behavior:

### Exemplary Subloops at 50 kHz

In [ ]:
# HB curves

fig, axs = plt.subplots(1, 5, figsize=(20, 6), squeeze=False)
fig.suptitle("Exemplary short trajectory per material for 50 kHz")

for idx, material_set in tqdm.tqdm(enumerate(dataset)):
    row_idx = 0 if idx < 5 else 1
    col_idx = idx % 5

    B_values = material_set[0].B[125, 1000:2000]
    H_values = material_set[0].H[125, 1000:2000]
    axs[row_idx, col_idx].plot(H_values, B_values)
    axs[row_idx, col_idx].set_title(material_set.material_name)
    
for ax in axs[0, :]:
    ax.set_ylabel("B in Vs/m^2")
    ax.set_xlabel("H in A/m") 

for ax_ in axs:
    for ax in ax_:
        ax.grid()

fig.tight_layout()
plt.show()

### Sequence starts (10 steps)

In [ ]:
# HB curves

for frequency in tqdm.tqdm(dataset[0].frequencies):
    fig, axs = plt.subplots(1, 5, figsize=(20, 6), squeeze=False)
    fig.suptitle("Hysteresis per material")
    
    for idx, material_set in enumerate(dataset):
        row_idx = 0 if idx < 5 else 1
        col_idx = idx % 5
    
        try:
            frequency_set = material_set.at_frequency(frequency) 
        except IndexError:
            continue

        for sequence_idx in range(frequency_set.B.shape[0]):
            B_values = frequency_set.B[sequence_idx, :10]
            H_values = frequency_set.H[sequence_idx, :10]
            axs[row_idx, col_idx].plot(H_values, B_values, alpha=.7)
    
        axs[row_idx, col_idx].set_title(material_set.material_name + " at " + str(frequency_set.frequency / 1e3) + " kHz")
        
    for ax in axs[0, :]:
        ax.set_ylabel("B in Vs/m^2")
        ax.set_xlabel("H in A/m")

    for ax_ in axs:
        for ax in ax_:
            ax.grid()
    
    fig.tight_layout()
    plt.show()

## Exemplary Time series:

In [ ]:
# H(t) and B(t) curves

for seq_idx, start, end in zip(jnp.arange(0, 10, 1), jnp.arange(0, 10_000, 1_000), jnp.arange(1_000, 11_000, 1_000)):

    fig, axs = plt.subplots(1, 5, figsize=(20, 6), squeeze=False)
    fig.suptitle("Exemplary short trajectory per material for 50 kHz")
    
    for idx, material_set in tqdm.tqdm(enumerate(dataset)):
        row_idx = 0 if idx < 5 else 1
        col_idx = idx % 5
    
        B_values = material_set[0].B[seq_idx, start:end]
        H_values = material_set[0].H[seq_idx, start:end]
        axs[row_idx, col_idx].plot(H_values, label="H")
        axs[row_idx, col_idx].plot(B_values, label="B")
        axs[row_idx, col_idx].set_title(material_set.material_name)
        
    
    for ax_ in axs:
        for ax in ax_:
            ax.grid()
            ax.legend()
    
    fig.tight_layout()
    plt.show()

In [ ]:
# Normalized H(t) and B(t) curves

for seq_idx, start, end in zip(jnp.arange(0, 10, 1), jnp.arange(0, 10_000, 1_000), jnp.arange(1_000, 11_000, 1_000)):

    fig, axs = plt.subplots(1, 5, figsize=(20, 6), squeeze=False)
    fig.suptitle("Exemplary short trajectory per material for 50 kHz")
    
    for idx, material_set in tqdm.tqdm(enumerate(dataset)):
        row_idx = 0 if idx < 5 else 1
        col_idx = idx % 5
    
        B_values = material_set[0].B[seq_idx, start:end]
        H_values = material_set[0].H[seq_idx, start:end]
        axs[row_idx, col_idx].plot(H_values / jnp.max(jnp.abs(H_values)), label="H")
        axs[row_idx, col_idx].plot(B_values / jnp.max(jnp.abs(B_values)), label="B")
        axs[row_idx, col_idx].set_title(material_set.material_name)
        
    
    for ax_ in axs:
        for ax in ax_:
            ax.grid()
            ax.legend()
    
    fig.tight_layout()
    plt.show()

In [ ]:
# Normalized H(t) and B(t) curves

for seq_idx, start, end in zip(jnp.arange(0, 10, 1), jnp.arange(0, 10_000, 1_000), jnp.arange(1_000, 11_000, 1_000)):

    fig, axs = plt.subplots(1, 5, figsize=(20, 6), squeeze=False)
    fig.suptitle("Exemplary short trajectory per material for 50 kHz")
    
    for idx, material_set in tqdm.tqdm(enumerate(dataset)):
        row_idx = 0 if idx < 5 else 1
        col_idx = idx % 5
    
        B_values = material_set[0].B[seq_idx, start:end]
        H_values = material_set[0].H[seq_idx, start:end]
        axs[row_idx, col_idx].plot(H_values / jnp.max(jnp.abs(material_set[0].H[seq_idx, :])), label="H")
        axs[row_idx, col_idx].plot(B_values / jnp.max(jnp.abs(material_set[0].B[seq_idx, :])), label="B")
        axs[row_idx, col_idx].set_title(material_set.material_name)
        
    
    for ax_ in axs:
        for ax in ax_:
            ax.grid()
            ax.legend()
    
    fig.tight_layout()
    plt.show()

- Sign changes are always reflected as sign changes in the other component, i.e., a sign change in B always occurs together with a sign change in H.
- It seems that B reacts on the changes in B after the change has already occured
- **you have to be able to make simplifications/regularizations based on this**
- Is it easier to predict $\Delta H$ instead of the full value?

## Subsampling:
- the signals are heavily oversampled and can be drastically reduced without loss of information

In [ ]:
# # Normalized H(t) and B(t) curves

# raise ValueError("time axis seems off")

# tau = 1 / (16 * 10^6)

# for frequency in tqdm.tqdm(dataset[0].frequencies):
#     for seq_idx, start, end in zip(jnp.arange(0, 2, 1), jnp.arange(0, 300, 100), jnp.arange(300, 600, 100)):
    
#         fig, axs = plt.subplots(1, 5, figsize=(20, 6), squeeze=False)
#         fig.suptitle(f"Exemplary short trajectory per material for {frequency} kHz")
        
#         for idx, material_set in tqdm.tqdm(enumerate(dataset)):
#             row_idx = 0 if idx < 5 else 1
#             col_idx = idx % 5

#             frequency_set = material_set.at_frequency(frequency)

#             subsampling_freq = 10
#             t = np.linspace(0, (end - start - 1) * tau, int((end - start) / subsampling_freq))

#             B_values = frequency_set.B[seq_idx, start:end:subsampling_freq]
#             H_values = frequency_set.H[seq_idx, start:end:subsampling_freq]
#             axs[row_idx, col_idx].plot(t, H_values / jnp.max(jnp.abs(frequency_set.H[seq_idx, :])), label="H_subsampled")
#             axs[row_idx, col_idx].plot(t, B_values / jnp.max(jnp.abs(frequency_set.B[seq_idx, :])), label="B_subsampled")
#             axs[row_idx, col_idx].set_title(material_set.material_name)

#             subsampling_freq = 1
#             t = np.linspace(0, (end - start -1) * tau, int((end - start) / subsampling_freq))
            
#             B_values = frequency_set.B[seq_idx, start:end:subsampling_freq]
#             H_values = frequency_set.H[seq_idx, start:end:subsampling_freq]
#             axs[row_idx, col_idx].plot(t, H_values / jnp.max(jnp.abs(frequency_set.H[seq_idx, :])), label="H")
#             axs[row_idx, col_idx].plot(t, B_values / jnp.max(jnp.abs(frequency_set.B[seq_idx, :])), label="B")
#             axs[row_idx, col_idx].set_title(material_set.material_name)
            
        
#         for ax_ in axs:
#             for ax in ax_:
#                 ax.grid()
#                 ax.legend()
        
#         fig.tight_layout()
#         plt.show()